In [1]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:4 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Hit:7 http://archive.ubuntu.com/ubuntu focal InRelease
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:14 http://ppa.launchpad.net/ubuntugis/p

In [2]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-03-07 14:25:03--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.48MB/s    in 0.2s    

2023-03-07 14:25:03 (5.48 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [4]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Grocery_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Grocery_v1_00.tsv.gz"), sep="\t", header=True)

# Show DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42521656|R26MV8D0KG6QI6|B000SAQCWC|     159713740|The Cravings Plac...|         Grocery|          5|            0|          0|   N|                Y|Using these for y...|As a family aller...| 2015-08-31|
|         US|   12049833|R1OF8GP57AQ1A0|B00509LVIQ|     138680402|Mauna Loa Macadam...|         Grocery|          5|    

In [5]:
# Get the number of rows in the DataFrame.
df.count()

2402458

# Transform the Data

## Create the "review_id_table".

In [6]:
from pyspark.sql.functions import to_date, col
# Create the "review_id_df" DataFrame with the appropriate columns and data types.
# Select the necessary columns
review_id_df = df.select("review_id", "customer_id", "product_id", "product_parent", to_date("review_date", "yyyy-MM-dd").alias("review_date"))
# Cast "customer_id" and "product_parent" column to integer
review_id_df = review_id_df.withColumn("customer_id", col("customer_id").cast("integer")).withColumn("product_parent", col("product_parent").cast("integer"))
# Show the resulting DataFrame
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R26MV8D0KG6QI6|   42521656|B000SAQCWC|     159713740| 2015-08-31|
|R1OF8GP57AQ1A0|   12049833|B00509LVIQ|     138680402| 2015-08-31|
|R3VDC1QB6MC4ZZ|     107642|B00KHXESLC|     252021703| 2015-08-31|
|R12FA3DCF8F9ER|    6042304|B000F8JIIC|     752728342| 2015-08-31|
| RTWHVNV6X4CNJ|   18123821|B004ZWR9RQ|     552138758| 2015-08-31|
| RIG9AWFOGRDVO|   23649464|B00AL6QBZ6|     681475449| 2015-08-31|
|R1S1XSG4ZCHDGS|   32778285|B00BCNSTRA|     578681693| 2015-08-31|
| RB15NBVY5ELVW|   46612941|B0089MM2BG|     350968436| 2015-08-31|
| R56358YM1ZJ7I|   31525399|B00Y1C9770|     729982780| 2015-08-31|
|R1ODXB3C9UP3NL|   19624355|B00J074W94|       2499702| 2015-08-31|
|R155ZMVLD5C9BP|   31910375|B0001VKKOO|     189938205| 2015-08-31|
|R2IXW43PUNYM0G|   12816533|B007TGH4CK|     398128262| 2015-08

## Create the "products" Table

In [38]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
products_df = df.select("product_id", "product_title").dropDuplicates()
# Filter out product_id "B004Buu488"
products_df = products_df.filter(df["product_id"] != "B004BUU488")
# Show the resulting DataFrame
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B0088PCJ1C|Bob's Pickle Pops...|
|B005P0U5BO|Envirokidz Organi...|
|B00K2RY8GI|Coffee Variety Sa...|
|B008233EI2|Once Again Organi...|
|B0029K0U90|Mrs. Wages ALL NA...|
|B003H4AMEU|Bhuja Crunchy Sea...|
|B004AJIG76|Krusteaz Belgian ...|
|B002LME6OQ|Prince of Peace O...|
|B00NP7U2U8|Yoki - Cheese Bre...|
|B00ISQD2YO|Nescafe with Coff...|
|B003X35RR2|Bob's Red Mill Gl...|
|B00WVSNJ8Y|Organic Valley, O...|
|B0002UM13C|Sour Cherry Prese...|
|B0042M7PS2|Rodelle Gourmet B...|
|B002DVPTVO|Coffee Go Candy -...|
|B008EL7N2U|Melitta Coffee Gr...|
|B00CCS8Y5Q|Trader Joe's Coco...|
|B006ZOXLF6|Flossugar (1/2 ga...|
|B004RJ0C1W|Camp Chicory & Co...|
|B005JJZ430|KONA BLEND COFFEE...|
+----------+--------------------+
only showing top 20 rows



## Create the "customers" Table

In [8]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
customers_df = df.groupBy("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count").withColumn("customer_id", col("customer_id").cast("integer"))
# Show the resulting DataFrame
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   25415089|            27|
|   18869981|             2|
|   51052113|            22|
|   13203460|             3|
|   30238476|             1|
|   33212984|             1|
|   37757447|            13|
|   51706294|             3|
|   51506042|             1|
|   31325270|             1|
|   21174138|             9|
|   44662810|             1|
|   33282498|             1|
|   22050232|             1|
|   22279960|             1|
|   46169082|             1|
|   48542530|             2|
|   52857540|             2|
|   46727040|             7|
|   42665845|             1|
+-----------+--------------+
only showing top 20 rows



## Create the "vine_table".

In [9]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
vine_df = df.select("review_id", "star_rating", "helpful_votes", "total_votes", "vine") \
            .withColumn("star_rating", col("star_rating").cast("integer")) \
            .withColumn("helpful_votes", col("helpful_votes").cast("integer")) \
            .withColumn("total_votes", col("total_votes").cast("integer"))
# Show the resulting DataFrame
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R26MV8D0KG6QI6|          5|            0|          0|   N|
|R1OF8GP57AQ1A0|          5|            0|          0|   N|
|R3VDC1QB6MC4ZZ|          5|            0|          0|   N|
|R12FA3DCF8F9ER|          5|            0|          0|   N|
| RTWHVNV6X4CNJ|          5|            0|          0|   N|
| RIG9AWFOGRDVO|          2|            1|          1|   N|
|R1S1XSG4ZCHDGS|          5|            1|          1|   N|
| RB15NBVY5ELVW|          5|            2|          2|   N|
| R56358YM1ZJ7I|          5|            0|          0|   N|
|R1ODXB3C9UP3NL|          1|            1|          3|   N|
|R155ZMVLD5C9BP|          4|            3|          4|   N|
|R2IXW43PUNYM0G|          4|            1|          1|   N|
|R3LYEG1QCK2BG0|          5|            0|          0|   N|
|R20LREICPM3YH0|          5|            

# Load

In [12]:
mode = "append"
jdbc_url="jdbc:postgresql://<endpoint>:5432/my_data_class_db"
config = {"user":"postgres", "password": "<password>", "driver":"org.postgresql.Driver"}

In [13]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [39]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode="overwrite", properties=config)

In [40]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [41]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)